In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import remaining libraries  

In [6]:
from sklearn.model_selection import train_test_split

## Load datasets  

In [7]:
traindf = pd.read_csv("data/train.csv")
traindf.shape

(162758, 5)

In [8]:
testdf = pd.read_csv("data/test.csv")
testdf.shape

(55315, 4)

## Getting data ready for training and evaluating models  

`Handle missing data. Current strategy is to fill in some custom text.`  
`Separate features and labels from traindf`  
`Split training data into training and testing parts`

In [9]:
traindf.shape

(162758, 5)

In [10]:
traindf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162758 entries, 0 to 162757
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   movieid             162758 non-null  object
 1   reviewerName        162758 non-null  object
 2   isFrequentReviewer  162758 non-null  bool  
 3   reviewText          156311 non-null  object
 4   sentiment           162758 non-null  object
dtypes: bool(1), object(4)
memory usage: 5.1+ MB


In [11]:
traindf.isna().sum()

movieid                  0
reviewerName             0
isFrequentReviewer       0
reviewText            6447
sentiment                0
dtype: int64

### Fill empty reviewText with custom text  

In [12]:
traindf["reviewText"].fillna("empty", inplace=True)
traindf.isna().sum()

movieid               0
reviewerName          0
isFrequentReviewer    0
reviewText            0
sentiment             0
dtype: int64

### Separate features and labels  

In [13]:
train_features = traindf["reviewText"]
train_labels = traindf.iloc[:, -1]
train_features.shape, train_labels.shape

((162758,), (162758,))

### Split traindf into training and testing parts  

In [14]:
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.25, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((122068,), (40690,), (122068,), (40690,))

### Needs further exploration  

`stratify based on reviewer type?`  
`play with test size`  

## Model evaluation plan and code  

In [15]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score

In [16]:
def evaluate(y_test, y_pred):
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    # print(f1_score(y_test, y_pred))
    ConfusionMatrixDisplay(y_test, y_pred)
    return

# Data preprocessing  

## Define stop words  

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer

In [18]:
stop_words = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"]
type(stop_words), len(stop_words)

(list, 1160)

## Feature extraction  

### Understand CountVectorizer and TfidfVectorizer  

In [19]:
# text = X_train.copy()
# text.head()

In [20]:
# vectorizer = TfidfVectorizer(stop_words=stop_words)
# vectorized_text = vectorizer.fit_transform(text["reviewText"])
# vectorized_text

`Without stop words`  
<122068x58624 sparse matrix of type '<class 'numpy.float64'>'
	with 2312222 stored elements in Compressed Sparse Row format>

`With stop words`  
<122068x57784 sparse matrix of type '<class 'numpy.float64'>'
	with 1188395 stored elements in Compressed Sparse Row format>   

In [21]:
# vectorizer2 = CountVectorizer(stop_words=stop_words)
# vectorized_text2 = vectorizer2.fit_transform(text["reviewText"])
# vectorized_text2

`Without stop words`  
<122068x58624 sparse matrix of type '<class 'numpy.int64'>'
	with 2312222 stored elements in Compressed Sparse Row format>

`With stop words`  
<122068x57784 sparse matrix of type '<class 'numpy.int64'>'
	with 1188395 stored elements in Compressed Sparse Row format>

### Conclusion   
`TfidfVectorizer is equivalent to CountVectorizer followed by TfidfTransformer.`  

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

## Model building workflow  

`Make pipeline for preprocesing and model training and predictions`  


In [22]:
from sklearn.pipeline import Pipeline

In [23]:
def preprocess_n_train(X_train, y_train, preprocessor, model):
    pipe = Pipeline(steps=[
                            ("preprocessor", preprocessor),
                            ("model", model)
                        ])
    
    pipe.fit(X_train, y_train)

    print("Given model has been trained. Use predict method to get predictions array.")
    return pipe


def predict_n_evaluate(pipeline, X_test, y_test):
    y_pred = pipeline.predict(X_test)
    print(f"y_pred shape: {y_pred.shape}")
    print(f"Summary of predictions: {np.unique(y_pred, return_counts=True)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    return y_pred

# def evaluate(y_test, y_pred):
#     print(classification_report(y_test, y_pred))
#     print(confusion_matrix(y_test, y_pred))
#     # print(f1_score(y_test, y_pred))
#     ConfusionMatrixDisplay(y_test, y_pred)
#     return

## Preprocessor(s)  

In [60]:
tvec = TfidfVectorizer()
tvec_sw = TfidfVectorizer(stop_words=stop_words)

hvec = HashingVectorizer()

## Logistic Regression with TfidVectorizer for preprocessing  

In [25]:
from sklearn.linear_model import LogisticRegression

In [111]:
logreg = LogisticRegression(max_iter=100000)
logreg

LogisticRegression(max_iter=100000)

In [27]:
tvec = TfidfVectorizer()
cvec = CountVectorizer()        # Note that TfidfVectorizer and CountVectorizer+TfidsTransformer do the same function
tvec, cvec

(TfidfVectorizer(), CountVectorizer())

In [112]:
logreg_pipe = preprocess_n_train(X_train, y_train, tvec, logreg)
logreg_pipe

Given model has been trained. Use predict method to get predictions array.


Pipeline(steps=[('preprocessor', TfidfVectorizer()),
                ('model', LogisticRegression(max_iter=100000))])

In [113]:
y_pred_logreg = predict_n_evaluate(logreg_pipe, X_test, y_test)
y_pred_logreg

y_pred shape: (40690,)
Summary of predictions: (array(['NEGATIVE', 'POSITIVE'], dtype=object), array([10012, 30678], dtype=int64))
              precision    recall  f1-score   support

    NEGATIVE       0.76      0.57      0.65     13430
    POSITIVE       0.81      0.91      0.86     27260

    accuracy                           0.80     40690
   macro avg       0.79      0.74      0.75     40690
weighted avg       0.79      0.80      0.79     40690

[[ 7619  5811]
 [ 2393 24867]]


array(['POSITIVE', 'NEGATIVE', 'NEGATIVE', ..., 'POSITIVE', 'POSITIVE',
       'POSITIVE'], dtype=object)

### Logistic Regression with stop words  

In [30]:
tvec_sw = TfidfVectorizer(stop_words=stop_words)

In [31]:
logreg_sw_pipe = preprocess_n_train(X_train, y_train, tvec_sw, logreg)
logreg_sw_pipe

c:\Users\All users\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['articl', 'mon'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Given model has been trained. Use predict method to get predictions array.


Pipeline(steps=[('preprocessor',
                 TfidfVectorizer(stop_words=['0o', '0s', '3a', '3b', '3d', '6b',
                                             '6o', 'a', 'a1', 'a2', 'a3', 'a4',
                                             'ab', 'able', 'about', 'above',
                                             'abst', 'ac', 'accordance',
                                             'according', 'accordingly',
                                             'across', 'act', 'actually', 'ad',
                                             'added', 'adj', 'ae', 'af',
                                             'affected', ...])),
                ('model', LogisticRegression(max_iter=1000))])

In [32]:
y_pred_logreg_sw = predict_n_evaluate(logreg_sw_pipe, X_test, y_test)
y_pred_logreg_sw

y_pred shape: (40690,)
Summary of predictions: (array(['NEGATIVE', 'POSITIVE'], dtype=object), array([ 9168, 31522], dtype=int64))
              precision    recall  f1-score   support

    NEGATIVE       0.76      0.52      0.62     13430
    POSITIVE       0.79      0.92      0.85     27260

    accuracy                           0.79     40690
   macro avg       0.78      0.72      0.73     40690
weighted avg       0.78      0.79      0.77     40690

[[ 6965  6465]
 [ 2203 25057]]


array(['POSITIVE', 'NEGATIVE', 'POSITIVE', ..., 'POSITIVE', 'POSITIVE',
       'POSITIVE'], dtype=object)

### Logistic regression with different ngram ranges  

#### ngram range (1,2)  

In [49]:
tvec_ngram_2 = TfidfVectorizer(ngram_range=(1,2))

In [50]:
logreg_ng2_pipe = preprocess_n_train(X_train, y_train, tvec_ngram_2, logreg)
logreg_ng2_pipe

Given model has been trained. Use predict method to get predictions array.


Pipeline(steps=[('preprocessor', TfidfVectorizer(ngram_range=(1, 2))),
                ('model', LogisticRegression(max_iter=1000))])

In [51]:
y_pred_logreg_ng2 = predict_n_evaluate(logreg_ng2_pipe, X_test, y_test)
y_pred_logreg_ng2

y_pred shape: (40690,)
Summary of predictions: (array(['NEGATIVE', 'POSITIVE'], dtype=object), array([ 9270, 31420], dtype=int64))
              precision    recall  f1-score   support

    NEGATIVE       0.79      0.54      0.64     13430
    POSITIVE       0.81      0.93      0.86     27260

    accuracy                           0.80     40690
   macro avg       0.80      0.74      0.75     40690
weighted avg       0.80      0.80      0.79     40690

[[ 7309  6121]
 [ 1961 25299]]


array(['POSITIVE', 'NEGATIVE', 'NEGATIVE', ..., 'POSITIVE', 'POSITIVE',
       'POSITIVE'], dtype=object)

#### ngram range (1,2) with stop words  

In [72]:
tvec_ngram_2_sw = TfidfVectorizer(ngram_range=(1,2), stop_words={'english'})

In [73]:
logreg_ng2_sw_pipe = preprocess_n_train(X_train, y_train, tvec_ngram_2_sw, logreg)
logreg_ng2_sw_pipe

Given model has been trained. Use predict method to get predictions array.


Pipeline(steps=[('preprocessor',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words={'english'})),
                ('model', LogisticRegression(max_iter=1000))])

In [74]:
y_pred_logreg_ng2_sw = predict_n_evaluate(logreg_ng2_sw_pipe, X_test, y_test)
y_pred_logreg_ng2_sw

y_pred shape: (40690,)
Summary of predictions: (array(['NEGATIVE', 'POSITIVE'], dtype=object), array([ 9268, 31422], dtype=int64))
              precision    recall  f1-score   support

    NEGATIVE       0.79      0.54      0.64     13430
    POSITIVE       0.81      0.93      0.86     27260

    accuracy                           0.80     40690
   macro avg       0.80      0.74      0.75     40690
weighted avg       0.80      0.80      0.79     40690

[[ 7309  6121]
 [ 1959 25301]]


array(['POSITIVE', 'NEGATIVE', 'NEGATIVE', ..., 'POSITIVE', 'POSITIVE',
       'POSITIVE'], dtype=object)

#### ngram range (1,2) with stop words with min_df and max_df manipulation  

In [147]:
tvec_ngram_2_sw_df = TfidfVectorizer(ngram_range=(1,2), stop_words=None, min_df=0.0001, max_df=0.5)
tvec_ngram_2_sw_df

TfidfVectorizer(max_df=0.5, min_df=0.0001, ngram_range=(1, 2))

In [148]:
logreg_ng2_sw_df_pipe = preprocess_n_train(X_train, y_train, tvec_ngram_2_sw_df, logreg)
logreg_ng2_sw_df_pipe

Given model has been trained. Use predict method to get predictions array.


Pipeline(steps=[('preprocessor',
                 TfidfVectorizer(max_df=0.5, min_df=0.0001,
                                 ngram_range=(1, 2))),
                ('model', LogisticRegression(max_iter=100000))])

In [149]:
y_pred_logreg_ng2_sw_df = predict_n_evaluate(logreg_ng2_sw_df_pipe, X_test, y_test)
y_pred_logreg_ng2_sw_df

y_pred shape: (40690,)
Summary of predictions: (array(['NEGATIVE', 'POSITIVE'], dtype=object), array([10065, 30625], dtype=int64))
              precision    recall  f1-score   support

    NEGATIVE       0.77      0.58      0.66     13430
    POSITIVE       0.82      0.92      0.86     27260

    accuracy                           0.81     40690
   macro avg       0.79      0.75      0.76     40690
weighted avg       0.80      0.81      0.80     40690

[[ 7788  5642]
 [ 2277 24983]]


array(['POSITIVE', 'NEGATIVE', 'NEGATIVE', ..., 'POSITIVE', 'POSITIVE',
       'POSITIVE'], dtype=object)

#### ngram range (1,3)  

In [52]:
tvec_ngram_3 = TfidfVectorizer(ngram_range=(1,3))
tvec_ngram_3

TfidfVectorizer(ngram_range=(1, 3))

In [53]:
logreg_ng3_pipe = preprocess_n_train(X_train, y_train, tvec_ngram_3, logreg)
logreg_ng3_pipe

Given model has been trained. Use predict method to get predictions array.


Pipeline(steps=[('preprocessor', TfidfVectorizer(ngram_range=(1, 3))),
                ('model', LogisticRegression(max_iter=1000))])

In [55]:
y_pred_logreg_ng3 = predict_n_evaluate(logreg_ng3_pipe, X_test, y_test)
y_pred_logreg_ng3

y_pred shape: (40690,)
Summary of predictions: (array(['NEGATIVE', 'POSITIVE'], dtype=object), array([ 8769, 31921], dtype=int64))
              precision    recall  f1-score   support

    NEGATIVE       0.79      0.51      0.62     13430
    POSITIVE       0.79      0.93      0.86     27260

    accuracy                           0.79     40690
   macro avg       0.79      0.72      0.74     40690
weighted avg       0.79      0.79      0.78     40690

[[ 6885  6545]
 [ 1884 25376]]


array(['POSITIVE', 'NEGATIVE', 'NEGATIVE', ..., 'POSITIVE', 'POSITIVE',
       'POSITIVE'], dtype=object)

### Logistic Regression with HashingVectorizer  

In [61]:
hvec = HashingVectorizer()

In [62]:
logreg_hvec_pipe = preprocess_n_train(X_train, y_train, hvec, logreg)
logreg_hvec_pipe

Given model has been trained. Use predict method to get predictions array.


Pipeline(steps=[('preprocessor', HashingVectorizer()),
                ('model', LogisticRegression(max_iter=1000))])

In [63]:
y_pred_logreg_hvec = predict_n_evaluate(logreg_hvec_pipe, X_test, y_test)
y_pred_logreg_hvec

y_pred shape: (40690,)
Summary of predictions: (array(['NEGATIVE', 'POSITIVE'], dtype=object), array([ 9489, 31201], dtype=int64))
              precision    recall  f1-score   support

    NEGATIVE       0.75      0.53      0.62     13430
    POSITIVE       0.80      0.91      0.85     27260

    accuracy                           0.79     40690
   macro avg       0.78      0.72      0.74     40690
weighted avg       0.78      0.79      0.78     40690

[[ 7159  6271]
 [ 2330 24930]]


array(['POSITIVE', 'NEGATIVE', 'NEGATIVE', ..., 'POSITIVE', 'POSITIVE',
       'POSITIVE'], dtype=object)

### with ngram_range (1,2)  

In [64]:
hvec_ng2 = HashingVectorizer(ngram_range=(1,2))
hvec_ng2

HashingVectorizer(ngram_range=(1, 2))

In [65]:
logreg_hvec_ng2_pipe = preprocess_n_train(X_train, y_train, hvec_ng2, logreg)
logreg_hvec_ng2_pipe

Given model has been trained. Use predict method to get predictions array.


Pipeline(steps=[('preprocessor', HashingVectorizer(ngram_range=(1, 2))),
                ('model', LogisticRegression(max_iter=1000))])

In [66]:
y_pred_logreg_hvec_ng2 = predict_n_evaluate(logreg_hvec_ng2_pipe, X_test, y_test)
y_pred_logreg_hvec_ng2

y_pred shape: (40690,)
Summary of predictions: (array(['NEGATIVE', 'POSITIVE'], dtype=object), array([ 9281, 31409], dtype=int64))
              precision    recall  f1-score   support

    NEGATIVE       0.77      0.53      0.63     13430
    POSITIVE       0.80      0.92      0.86     27260

    accuracy                           0.79     40690
   macro avg       0.79      0.73      0.74     40690
weighted avg       0.79      0.79      0.78     40690

[[ 7153  6277]
 [ 2128 25132]]


array(['POSITIVE', 'NEGATIVE', 'NEGATIVE', ..., 'POSITIVE', 'POSITIVE',
       'POSITIVE'], dtype=object)

### with stop words ('english')  

In [67]:
hvec_ng2_sw = HashingVectorizer(ngram_range=(1,2), stop_words={'english'})
hvec_ng2_sw

HashingVectorizer(ngram_range=(1, 2), stop_words={'english'})

In [70]:
logreg_hvec_ng2_sw_pipe = preprocess_n_train(X_train, y_train, hvec_ng2_sw, logreg)
logreg_hvec_ng2_sw_pipe

Given model has been trained. Use predict method to get predictions array.


Pipeline(steps=[('preprocessor',
                 HashingVectorizer(ngram_range=(1, 2), stop_words={'english'})),
                ('model', LogisticRegression(max_iter=1000))])

In [71]:
y_pred_logreg_hvec_ng2_sw = predict_n_evaluate(logreg_hvec_ng2_sw_pipe, X_test, y_test)
y_pred_logreg_hvec_ng2_sw

y_pred shape: (40690,)
Summary of predictions: (array(['NEGATIVE', 'POSITIVE'], dtype=object), array([ 9278, 31412], dtype=int64))
              precision    recall  f1-score   support

    NEGATIVE       0.77      0.53      0.63     13430
    POSITIVE       0.80      0.92      0.86     27260

    accuracy                           0.79     40690
   macro avg       0.79      0.73      0.74     40690
weighted avg       0.79      0.79      0.78     40690

[[ 7149  6281]
 [ 2129 25131]]


array(['POSITIVE', 'NEGATIVE', 'NEGATIVE', ..., 'POSITIVE', 'POSITIVE',
       'POSITIVE'], dtype=object)

### Logistic Regression with stop words with polynomial features  

In [33]:
from sklearn.preprocessing import PolynomialFeatures
from  sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion 

In [47]:
tvec_sw_poly = FeatureUnion([
                            ("TfidsVectorizer", TfidfVectorizer()),
                            ("PolynomialFeatures", PolynomialFeatures(degree=2, interaction_only=True), )
                            ])
tvec_sw_poly

FeatureUnion(transformer_list=[('TfidsVectorizer', TfidfVectorizer()),
                               ('PolynomialFeatures',
                                PolynomialFeatures(interaction_only=True))])

In [48]:
logreg_sw_poly_pipe = preprocess_n_train(X_train, y_train, tvec_sw_poly, logreg)
logreg_sw_poly_pipe

ValueError: could not convert string to float: 'It narrowly misses being a list of fan winks.  [Full review in Spanish] '

## SVM model  

In [35]:
from sklearn.svm import LinearSVC

In [36]:
svm = LinearSVC()
svm

LinearSVC()

In [37]:
svm_pipe = preprocess_n_train(X_train, y_train, tvec, svm)
svm_pipe

Given model has been trained. Use predict method to get predictions array.


Pipeline(steps=[('preprocessor', TfidfVectorizer()), ('model', LinearSVC())])

In [38]:
y_pred_svm = predict_n_evaluate(svm_pipe, X_test, y_test)
y_pred_svm

y_pred shape: (40690,)
Summary of predictions: (array(['NEGATIVE', 'POSITIVE'], dtype=object), array([11525, 29165], dtype=int64))
              precision    recall  f1-score   support

    NEGATIVE       0.72      0.62      0.66     13430
    POSITIVE       0.82      0.88      0.85     27260

    accuracy                           0.79     40690
   macro avg       0.77      0.75      0.76     40690
weighted avg       0.79      0.79      0.79     40690

[[ 8284  5146]
 [ 3241 24019]]


array(['NEGATIVE', 'NEGATIVE', 'POSITIVE', ..., 'POSITIVE', 'POSITIVE',
       'POSITIVE'], dtype=object)

### SVM model with stop words  

In [39]:
svm_sw_pipe = preprocess_n_train(X_train, y_train, tvec_sw, svm)
svm_sw_pipe

Given model has been trained. Use predict method to get predictions array.


Pipeline(steps=[('preprocessor',
                 TfidfVectorizer(stop_words=['0o', '0s', '3a', '3b', '3d', '6b',
                                             '6o', 'a', 'a1', 'a2', 'a3', 'a4',
                                             'ab', 'able', 'about', 'above',
                                             'abst', 'ac', 'accordance',
                                             'according', 'accordingly',
                                             'across', 'act', 'actually', 'ad',
                                             'added', 'adj', 'ae', 'af',
                                             'affected', ...])),
                ('model', LinearSVC())])

In [40]:
y_pred_svm_sw = predict_n_evaluate(svm_sw_pipe, X_test, y_test)
y_pred_svm_sw

y_pred shape: (40690,)
Summary of predictions: (array(['NEGATIVE', 'POSITIVE'], dtype=object), array([11111, 29579], dtype=int64))
              precision    recall  f1-score   support

    NEGATIVE       0.70      0.58      0.63     13430
    POSITIVE       0.81      0.88      0.84     27260

    accuracy                           0.78     40690
   macro avg       0.75      0.73      0.74     40690
weighted avg       0.77      0.78      0.77     40690

[[ 7760  5670]
 [ 3351 23909]]


array(['NEGATIVE', 'NEGATIVE', 'POSITIVE', ..., 'POSITIVE', 'POSITIVE',
       'POSITIVE'], dtype=object)

## Naive Bayes model  

### MultinomialNB  

In [41]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, CategoricalNB, ComplementNB, MultinomialNB

In [42]:
mnb = MultinomialNB()
mnb

MultinomialNB()

In [43]:
mnb_pipe = preprocess_n_train(X_train, y_train, tvec, mnb)
mnb_pipe

Given model has been trained. Use predict method to get predictions array.


Pipeline(steps=[('preprocessor', TfidfVectorizer()),
                ('model', MultinomialNB())])

In [44]:
y_pred_mnb = predict_n_evaluate(mnb_pipe, X_test, y_test)
y_pred_mnb

y_pred shape: (40690,)
Summary of predictions: (array(['NEGATIVE', 'POSITIVE'], dtype='<U8'), array([ 3936, 36754], dtype=int64))
              precision    recall  f1-score   support

    NEGATIVE       0.88      0.26      0.40     13430
    POSITIVE       0.73      0.98      0.84     27260

    accuracy                           0.74     40690
   macro avg       0.80      0.62      0.62     40690
weighted avg       0.78      0.74      0.69     40690

[[ 3467  9963]
 [  469 26791]]


array(['POSITIVE', 'NEGATIVE', 'POSITIVE', ..., 'POSITIVE', 'POSITIVE',
       'POSITIVE'], dtype='<U8')

### GaussianNB  

https://stackoverflow.com/questions/28384680/scikit-learns-pipeline-a-sparse-matrix-was-passed-but-dense-data-is-required

In [45]:
gnb = GaussianNB()
gnb

GaussianNB()

In [46]:
gnb_pipe = preprocess_n_train(X_train, y_train, tvec, gnb)
gnb_pipe

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

## Submitting predictions to Kaggle competition  

In [ ]:
# def submit(y_pred):
#     pred_df = pd.DataFrame(y_pred)
#     pred_df.columns = ["sentiment"]
#     pred_df.index.name = "id"
#     pred_df.to_csv("submission.csv")
#     return "Successfully created the submission file!!!"

In [ ]:
# submit(y_pred)